In [1]:
import os
import pandas as pd
import numpy as np
import seaborn
import bokeh
import sqlite3

In [2]:
pic = pd.read_excel("data/aviation_seats_flights_pic.xlsx")
# con = sqlite3.connect("test.db")
# pic.to_sql("pic", con=con)
pic.head(5)

,Country,ISO,Region,Date,Aircraft_type,Seats_arrivals_domestic,Seats_arrivals_interregional,Seats_arrivals_intraregional,Seats_arrivals_intl,Seats_arrivals_total,Available_seat_kilometers,Number_of_flights_domestic,Number_of_flights_interregional,Number_of_flights_intraregional,Number_of_flights_intl,Number_of_flights_total
0,Fiji,FJ,East Asia & Pacific,2019-01-01,passenger,839,273,3480,3753,4592,1.430416e+07,8,1,10,11,19
1,Fiji,FJ,East Asia & Pacific,2019-01-02,passenger,974,313,3471,3784,4758,1.495610e+07,8,1,10,11,19
2,Fiji,FJ,East Asia & Pacific,2019-01-03,passenger,1190,443,3675,4118,5308,1.592143e+07,10,2,12,14,24
3,Fiji,FJ,East Asia & Pacific,2019-01-04,passenger,831,586,3159,3745,4576,1.457334e+07,7,2,12,14,21
4,Fiji,FJ,East Asia & Pacific,2019-01-05,passenger,744,273,4752,5025,5769,1.773449e+07,7,1,12,13,20


In [3]:
def subset_data(data: pd.DataFrame,
                voi: str,
                condition: str):

    subset = (data[data[voi] == str(condition)]
              .reset_index()
              .drop("index", axis=1))

    return subset


def create_viz_df(data: pd.DataFrame,
                  start: str,
                  end: str,
                  voi: str):

    viz_df = pd.DataFrame(pd.date_range(
        start=start, end=end), columns=["Date"])
    for country in countries:
        country_df = (data[data.Country == str(country)]
                      [["Date", voi]]
                      .reset_index().drop("index", axis=1))
        country_df.columns = ["Date", str(country)]
        viz_df = viz_df.merge(country_df, how="left", on="Date")

    return viz_df

In [4]:
pic_tourism = subset_data(pic, "Aircraft_type", "passenger")
countries = pic.Country.unique().tolist()
psgfl = pic_tourism[["Country", "Date", "Seats_arrivals_intl", "Number_of_flights_intl"]]
viz_df = create_viz_df(psgfl, "2019-01-01", "2022-10-16", "Number_of_flights_intl")
viz_df["Date"] = pd.to_datetime(viz_df["Date"])

ma7 = viz_df.fillna(0).iloc[:, 1:].rolling(7).sum()
ma7["Date"] = pd.date_range(start="2019-01-01", end="2022-10-16")

In [108]:
month, week, year = list(), list(), list()
for idx in viz_df.index:
    week.append(viz_df["Date"][idx].week)
    month.append(viz_df["Date"][idx].month)
    year.append(viz_df["Date"][idx].year)

viz_df["week"], viz_df["month"], viz_df["year"] = week, month, year
Fiji_20_22= viz_df[["Date", "Fiji", "month", "year"]].groupby(by=["year", "month"]).sum().reset_index()[12:-2]


fiji = pd.read_csv("output/fj_num_visitors.csv")
fiji

,Unnamed: 0,year/month,month,visitors,total,year,dates
0,0,2020.0,January,65386,83624,2020,2020-01-01
1,1,NaN,February,46343,56791,2020,2020-02-01
2,2,NaN,March,27972,36138,2020,2020-03-01
3,3,NaN,April,678,1188,2020,2020-04-01
4,4,NaN,May,709,1119,2020,2020-05-01
5,5,NaN,June,413,1041,2020,2020-06-01
6,6,NaN,July,711,1486,2020,2020-07-01
7,7,NaN,August,976,1441,2020,2020-08-01
8,8,NaN,September,1005,1688,2020,2020-09-01
9,9,NaN,October,1125,2099,2020,2020-10-01


In [105]:
Fiji_20_22["Fiji"].corr(fiji["visitors"])
Fiji_20_22

,year,month,Fiji
12,2020,1,342.0
13,2020,2,248.0
14,2020,3,173.0
15,2020,4,9.0
16,2020,5,5.0
17,2020,6,9.0
18,2020,7,11.0
19,2020,8,15.0
20,2020,9,31.0
21,2020,10,44.0


In [29]:
from bokeh.plotting import show, figure, output_file
from bokeh.models import CustomJS, Select, ColumnDataSource, HoverTool, BoxZoomTool, ResetTool
from bokeh.layouts import row, column, Spacer

output_file("Viz/psg_19_22.html")

y_selector = Select(
    title="Country", options=countries, value="Country", width=100,
)

cds = ColumnDataSource(ma7)
render_cds = ColumnDataSource(
    {"x": ma7["Date"],
     "y": ma7["Fiji"],
     }
)

hover = HoverTool(tooltips=[('Date', '@x{%Y-%m-%d}'),
                            ('Number', '@y')],
                  formatters={'@x': 'datetime'})

p = figure(
    height=555, width=740,
    title="Number of International Passengers in 7 Days",
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label="Number of Flights",
    tools=[hover, BoxZoomTool(), ResetTool()],
)

line = p.line(source=render_cds, x="x", y="y", line_width=1, color="green")

jscode = """
// New data
render_cds.data['y'] = cds.data[y_selector.value];
yaxis[0].axis_label = y_selector.value;
render_cds.change.emit();
"""

args = dict(
    render_cds=render_cds,
    cds=cds,
    y_selector=y_selector,
    yaxis=p.yaxis,
)
y_selector.js_on_change("value", CustomJS(code=jscode, args=args))

layout = row(
    column(y_selector, Spacer(height=30), p),
)


show(layout)

In [42]:
for country in countries:
    country_df = (psgfl[psgfl.Country == str(country)]
                      .reset_index()
                      .drop("index", axis=1))
    date_df = pd.DataFrame(pd.date_range(start="2019-01-01", end="2022-10-16"), columns=["Date"])
    country_df = date_df.merge(country_df, how="left", on="Date")
    country_df["Country"] = str(country)
    country_df = country_df.fillna(0)
    country_df.to_csv("data/country/" + str(country).strip() + ".csv", encoding="utf-8")